In [10]:
import pyomo.environ as pyo
from pyomo.network import Arc
import numpy as np
from scipy import stats
from pyDOE2 import fullfact
import pandas as pd

from idaes.core import (
    Component,
    declare_process_block_class,
    FlowsheetBlock,
    MaterialBalanceType,
    MaterialFlowBasis,
    Phase,
    PhysicalParameterBlock,
    StateBlock,
    StateBlockData,
)
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import fix_state_vars
from idaes.models.unit_models import MSContactor, MSContactorInitializer
from idaes.models.unit_models import Mixer, MixingType, MomentumMixingType, MixerInitializer
from idaes.core.util.initialization import propagate_state

In [11]:
class _StateBlock(StateBlock):
    def fix_initialization_states(self):
        fix_state_vars(self)
    
    def initialization_routine(self):
        pass

In [12]:
@declare_process_block_class("LiCoStateBlock", block_class=_StateBlock)
class LiCoStateBlock1Data(StateBlockData):

    def build(self):
        super().build()

        self.flow_vol = pyo.Var(
            units=pyo.units.m**3 / pyo.units.hour,
            bounds=(1e-8, None),
        )
        self.conc_mass_solute = pyo.Var(
            ["Li", "Co"],
            units=pyo.units.kg / pyo.units.m**3,
            bounds=(1e-8, None),
        )

    def get_material_flow_terms(self, p, j):
        if j == "solvent":
            # Assume constant density of pure water
            return self.flow_vol * self.params.dens_H2O
        else:
            return self.flow_vol * self.conc_mass_solute[j]

    def get_material_flow_basis(self):
        return MaterialFlowBasis.mass

    def define_state_vars(self):
        return {
            "flow_vol": self.flow_vol,
            "conc_mass_solute": self.conc_mass_solute,
        }

In [13]:
@declare_process_block_class("LiCoParameters")
class LiCoParameterData(PhysicalParameterBlock):
    def build(self):
        super().build()

        self.phase1 = Phase()

        self.solvent = Component()
        self.Li = Component()
        self.Co = Component()
        
        self.dens_H2O = pyo.Param(
            default=1000,
            units=pyo.units.kg / pyo.units.m**3,
        )

        self._state_block_class = LiCoStateBlock

    @classmethod
    def define_metadata(cls, obj):
        obj.add_default_units(
            {
                "time": pyo.units.hour,
                "length": pyo.units.m,
                "mass": pyo.units.kg,
                "amount": pyo.units.mol,
                "temperature": pyo.units.K,
            }
        )

In [34]:
def solve_model(Q_fd, Li_fd, Co_fd, Q_df, Li_df=0, Co_df=0):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    m.fs.properties = LiCoParameters()

    # stage 1
    m.fs.stage1 = MSContactor(
        number_of_finite_elements=10,
        streams={
            "retentate": {
                "property_package": m.fs.properties,
                "has_energy_balance": False,
                "has_pressure_balance": False,
            },
            "permeate": {
                "property_package": m.fs.properties,
                "has_feed": False,
                "has_energy_balance": False,
                "has_pressure_balance": False,
            },
        },
    )
    
    # defining parameters
    J = 0.1 * pyo.units.m / pyo.units.hour
    w = 1.5 * pyo.units.m

    m.fs.solutes = pyo.Set(initialize=["Li", "Co"])
    m.fs.sieving_coefficient = pyo.Var(m.fs.solutes, units=pyo.units.dimensionless)
    m.fs.sieving_coefficient["Li"].fix(1.3)
    m.fs.sieving_coefficient["Co"].fix(0.5)

    m.fs.stage1.length = pyo.Var(units=pyo.units.m)
    m.fs.stage1.length.fix(10)

    # mass transfer equations
    def solvent_rule(b, s):
        return (
            b.material_transfer_term[0, s, "permeate", "retentate", "solvent"]
            == J * b.length * w * m.fs.properties.dens_H2O / 10
        )
    
    m.fs.stage1.solvent_flux = pyo.Constraint(
        m.fs.stage1.elements,
        rule=solvent_rule,
    )
    
    def solute_rule(b, s, j):
        if s == 1:
            in_state = b.retentate_inlet_state[0]
        else:
            sp = b.elements.prev(s)
            in_state = b.retentate[0, sp]
    
        return (
            pyo.log(b.retentate[0, s].conc_mass_solute[j])
            + (m.fs.sieving_coefficient[j] - 1)
            * pyo.log(in_state.flow_vol)
            == pyo.log(in_state.conc_mass_solute[j])
            + (m.fs.sieving_coefficient[j] - 1)
            * pyo.log(b.retentate[0, s].flow_vol)
        )
    
    m.fs.stage1.solute_sieving = pyo.Constraint(
        m.fs.stage1.elements,
        m.fs.solutes,
        rule=solute_rule,
    )

    # initial guess for stage 1 recycle (retentate of stage 3)
    m.fs.stage1.retentate_inlet.flow_vol[0].fix(Q_fd)
    m.fs.stage1.retentate_inlet.conc_mass_solute[0, "Li"].fix(Li_fd)
    m.fs.stage1.retentate_inlet.conc_mass_solute[0, "Co"].fix(Co_fd)

    # initializing first stage
    initializer = MSContactorInitializer()
    initializer.initialize(m.fs.stage1)

    # stage 2
    m.fs.stage2 = MSContactor(
        number_of_finite_elements=10,
        streams={
            "retentate":{
                "property_package": m.fs.properties,
                "has_energy_balance": False,
                "has_pressure_balance": False
            },
            "permeate":{
                "property_package": m.fs.properties,
                "has_feed": False,
                "has_energy_balance": False,
                "has_pressure_balance": False
            }        
        }
    )
    
    m.fs.stage2.length = pyo.Var(units=pyo.units.m)
    m.fs.stage2.length.fix(10)

    m.fs.stage2.solvent_flux = pyo.Constraint(m.fs.stage2.elements, rule=solvent_rule)
    m.fs.stage2.solute_sieving = pyo.Constraint(m.fs.stage2.elements, m.fs.solutes, rule=solute_rule)

    # mixer for permeate of stage 1 and retentate of stage 2
    m.fs.mix1 = Mixer(
        num_inlets=2,
        property_package=m.fs.properties,
        material_balance_type=MaterialBalanceType.componentTotal,
        energy_mixing_type=MixingType.none,
        momentum_mixing_type=MomentumMixingType.none,
    )
    
    m.fs.stream1 = Arc(
        source=m.fs.stage1.permeate_outlet,
        destination=m.fs.mix1.inlet_2,
    )
    
    m.fs.stream2 = Arc(
        source=m.fs.mix1.outlet,
        destination=m.fs.stage2.retentate_inlet,
    )
    
    m.fs.stream3 = Arc(
        source=m.fs.stage2.retentate_outlet,
        destination=m.fs.stage1.retentate_inlet,
    )

    # converting connections to contraints
    pyo.TransformationFactory("network.expand_arcs").apply_to(m)

    # unfixing stage 1 retentate inlet and fixing mixer 1 inlet_1
    m.fs.stage1.retentate_inlet.flow_vol[0].unfix()
    m.fs.stage1.retentate_inlet.conc_mass_solute[0, "Li"].unfix()
    m.fs.stage1.retentate_inlet.conc_mass_solute[0, "Co"].unfix()
    
    m.fs.mix1.inlet_1.flow_vol[0].fix(Q_fd)
    m.fs.mix1.inlet_1.conc_mass_solute[0, "Li"].fix(Li_fd)
    m.fs.mix1.inlet_1.conc_mass_solute[0, "Co"].fix(Co_fd)

    # copying stage 1 permeate outlet conditions to mixer 1 inlet_1
    propagate_state(
        destination=m.fs.mix1.inlet_2,
        source=m.fs.stage1.permeate_outlet,
    )

    # initializing mixer 1
    mix_initializer = MixerInitializer()
    mix_initializer.initialize(m.fs.mix1)

    # copying mixer 1 outlet conditions to stage 2 retentate inlet
    propagate_state(
        source=m.fs.mix1.outlet,
        destination=m.fs.stage2.retentate_inlet
    )

    # initializing stage 2
    initializer.initialize(m.fs.stage2)

    # stage 3
    m.fs.stage3 = MSContactor(
        number_of_finite_elements=10,
        streams={
            "retentate":{
                "property_package":m.fs.properties,
                "has_energy_balance":False,
                "has_pressure_balance":False,
                "side_streams":[10]
            },
            "permeate":{
                "property_package":m.fs.properties,
                "has_feed":False,
                "has_energy_balance":False,
                "has_pressure_balance":False
            }
        }
    )

    assert isinstance(m.fs.stage3, MSContactor)  # check that stage3 exists and is an MSContactor
    
    # Retentate side checks
    assert hasattr(m.fs.stage3, "retentate_inlet_state")  # check that there is a retentate feed
    assert hasattr(m.fs.stage3, "retentate_side_stream_state")  # check that a side stream exists
    for k in m.fs.stage3.retentate_side_stream_state:
        assert k == (0, 10)  # check that the side stream only exists at element 10
    assert not hasattr(m.fs.stage3, "retentate_energy_balance")  # check that there are no energy balances
    assert not hasattr(m.fs.stage3, "retentate_pressure_balance")  # check that there are no pressure balances
    
    # Permeate side checks
    assert not hasattr(m.fs.stage3, "permeate_inlet_state")  # check that there is no permeate feed
    assert not hasattr(m.fs.stage3, "permeate_side_stream_state")  # check that there are no side streams on permeate side
    assert not hasattr(m.fs.stage3, "permeate_energy_balance")  # check that there are no energy balances
    assert not hasattr(m.fs.stage3, "permeate_pressure_balance")  # check that there are no pressure balances

    m.fs.stage3.length = pyo.Var(units=pyo.units.m)
    m.fs.stage3.length.fix(10)
    
    def stage3_solute_rule(b, s, j):
        if s == 1:
            q_in = b.retentate_inlet_state[0].flow_vol
            c_in = b.retentate_inlet_state[0].conc_mass_solute[j]
        elif s == 10:
            sp = b.elements.prev(s)
            q_in = (
                b.retentate[0, sp].flow_vol
                + b.retentate_side_stream_state[0, 10].flow_vol
            )
            c_in = (
                b.retentate[0, sp].conc_mass_solute[j] * b.retentate[0, sp].flow_vol
                + b.retentate_side_stream_state[0, 10].conc_mass_solute[j]
                * b.retentate_side_stream_state[0, 10].flow_vol
            ) / q_in
        else:
            sp = b.elements.prev(s)
            q_in = b.retentate[0, sp].flow_vol
            c_in = b.retentate[0, sp].conc_mass_solute[j]
    
        return pyo.log(b.retentate[0, s].conc_mass_solute[j]) + (
            m.fs.sieving_coefficient[j] - 1
        ) * pyo.log(q_in) == pyo.log(c_in) + (m.fs.sieving_coefficient[j] - 1) * pyo.log(
            b.retentate[0, s].flow_vol
        )
    
    m.fs.stage3.solvent_flux = pyo.Constraint(
        m.fs.stage3.elements,
        rule=solvent_rule,
    )
    m.fs.stage3.solute_sieving = pyo.Constraint(
        m.fs.stage3.elements,
        m.fs.solutes,
        rule=stage3_solute_rule,
    )

    # mixer 2 for stage 2 permeate outlet and stage 3 retentate outlet
    m.fs.mix2 = Mixer(
        num_inlets=2,
        property_package=m.fs.properties,
        material_balance_type=MaterialBalanceType.componentTotal,
        energy_mixing_type=MixingType.none,
        momentum_mixing_type=MomentumMixingType.none
    )

    m.fs.stream4 = Arc(
        source=m.fs.stage2.permeate_outlet,
        destination=m.fs.mix2.inlet_2
    )
    
    m.fs.stream5 = Arc(
        source=m.fs.mix2.outlet,
        destination=m.fs.stage3.retentate_inlet
    )
    
    m.fs.stream6 = Arc(
        source=m.fs.stage3.retentate_outlet,
        destination=m.fs.mix1.inlet_1
    )
    
    pyo.TransformationFactory("network.expand_arcs").apply_to(m)

    # unfixing mixer 1 inlet_1
    m.fs.mix1.inlet_1.flow_vol[0].unfix()
    m.fs.mix1.inlet_1.conc_mass_solute[0, "Li"].unfix()
    m.fs.mix1.inlet_1.conc_mass_solute[0, "Co"].unfix()

    # adding and fixing diafiltrate stream to mixer 2 inlet_1
    m.fs.mix2.inlet_1.flow_vol[0].fix(Q_df)
    m.fs.mix2.inlet_1.conc_mass_solute[0, "Li"].fix(Li_df)
    m.fs.mix2.inlet_1.conc_mass_solute[0, "Co"].fix(Co_df)
    
    # fixing fresh feed conditions at element 10 of stage 3
    m.fs.stage3.retentate_side_stream_state[0, 10].flow_vol.fix(Q_fd)
    m.fs.stage3.retentate_side_stream_state[0, 10].conc_mass_solute["Li"].fix(Li_fd)
    m.fs.stage3.retentate_side_stream_state[0, 10].conc_mass_solute["Co"].fix(Co_fd)

    # copying stage 2 permeate outlet conditions to mixer 2 inlet_2
    propagate_state(
        source=m.fs.stage2.permeate_outlet,
        destination=m.fs.mix2.inlet_2
    )

    # initializing mixer 2
    mix_initializer.initialize(m.fs.mix2)

    # copying mixer 2 outlet conditions to stage 3 retentate inlet
    propagate_state(
        source=m.fs.mix2.outlet,
        destination=m.fs.stage3.retentate_inlet
    )

    # initializing stage 3
    initializer.initialize(m.fs.stage3)

    solver = pyo.SolverFactory("ipopt")
    solver.solve(m.fs)
    # m.pprint()

    # outlets result
    m.fs.stage3.permeate_outlet.display()
    m.fs.stage1.retentate_outlet.display()

    # lithium and cobalt recovery
    m.Li_recovery = pyo.Expression(
        expr=m.fs.stage3.permeate_outlet.flow_vol[0]
        * m.fs.stage3.permeate_outlet.conc_mass_solute[0, "Li"]
        / (
            m.fs.mix2.inlet_1.flow_vol[0]
            * m.fs.mix2.inlet_1.conc_mass_solute[0, "Li"]
            + m.fs.stage3.retentate_side_stream_state[0, 10].flow_vol
            * m.fs.stage3.retentate_side_stream_state[0, 10].conc_mass_solute["Li"]
        )
    )

    m.Co_recovery = pyo.Expression(
        expr=m.fs.stage1.retentate_outlet.flow_vol[0]
        * m.fs.stage1.retentate_outlet.conc_mass_solute[0, "Co"]
        / (
            m.fs.mix2.inlet_1.flow_vol[0]
            * m.fs.mix2.inlet_1.conc_mass_solute[0, "Co"]
            + m.fs.stage3.retentate_side_stream_state[0, 10].flow_vol
            * m.fs.stage3.retentate_side_stream_state[0, 10].conc_mass_solute["Co"]
        )
    )

    # Check recovery values
    m.Co_recovery.display()
    m.Li_recovery.display()

    # Increase stage length and re-solve
    m.fs.stage1.length.fix(756.4)
    m.fs.stage2.length.fix(756.4)
    m.fs.stage3.length.fix(756.4)
    
    solver.solve(m.fs)
    # m.pprint()
    
    m.Co_recovery.display()
    m.Li_recovery.display()

    return[pyo.value(m.fs.stage1.retentate_outlet.flow_vol[0]), pyo.value(m.fs.stage3.permeate_outlet.flow_vol[0]), 
           pyo.value(m.fs.stage1.retentate_outlet.conc_mass_solute[0, "Co"]), pyo.value(m.fs.stage3.permeate_outlet.conc_mass_solute[0, "Li"]),]

outlet = solve_model(100, 1.7, 17, 30)
print(outlet)

Q_feed = [90, 110]
Q_diaf = [27, 33]
Co_feed = [15, 19]
Li_feed = [1.5, 2.0]

# varing diafiltrate flowrate
# outlets = []
# for i in Q_diaf:
#     outlet_cond = solve_model(100, 1.7, 17, i)
#     outlets.append(outlet_cond)
# print(outlets)

# varying fresh feed flowrate
# outlets2 = []
# for i in Q_feed:
#     outlet_cond = solve_model(i, 1.7, 17, 30)
#     outlets2.append(outlet_cond)
# print(outlets2)

# # varying lithium concentration
# outlets3 = []
# for i in Li_feed:
#     outlet_cond = solve_model(100, i, 17, 30)
#     outlets3.append(outlet_cond)
# print(outlets3)

# # varying cobalt concentration
# outlets4 = []
# for i in Co_feed:
#     outlet_cond = solve_model(100, 1.7, i, 30)
#     outlets4.append(outlet_cond)
# print(outlets4)

# outlets5=[]
# for i in Q_diaf:
#     for j in Co_feed:
#         for l in Li_feed:
#             for r in Q_feed:
#                 outlet_cond = solve_model(r, l, j, i)
#                 outlets5.append(outlet_cond)
# print(outlets5)

WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,1].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,2].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,3].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,4].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,5].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting 

In [27]:
levels = [2, 2, 2, 2]
design_matrix = fullfact(levels)
factor_names = ["Q_fd", "Q_df", "Li_fd", "Co_fd"]
df = pd.DataFrame(design_matrix, columns=factor_names)
print(df)

inputs = []
outputs = []
def rescale(x, upper_bounds, lower_bounds):
    new_x = x*(upper_bounds - lower_bounds) + lower_bounds
    return(new_x)

for i in range(len(df["Q_fd"])):
    # Perform experiment with values
    exp_vals = df.iloc[i]

    # scale inputs
    Q_fd = rescale(exp_vals["Q_fd"], Q_feed[1], Q_feed[0])
    Q_df = rescale(exp_vals["Q_df"], Q_diaf[1], Q_diaf[0])
    Li_fd = rescale(exp_vals["Li_fd"], Li_feed[1], Li_feed[0])
    Co_fd = rescale(exp_vals["Co_fd"], Co_feed[1], Co_feed[0])

    inputs.append([Q_fd, Li_fd, Co_fd, Q_df])
    
    # Call model
    new_outputs = solve_model(Q_fd, Li_fd, Co_fd, Q_df)
    outputs.append(new_outputs)

input_names = ["Q_feed", "Li_feed", "Co_feed", "Q_diaf"]
output_names = ["Q_Co_prdt", "Q_Li_prdt", "C_Co_prdt", "C_Li_prdt"]
inputs_df = pd.DataFrame(inputs, columns=input_names)
outputs_df = pd.DataFrame(outputs, columns=output_names)

print(inputs_df)
print(outputs_df)

    Q_fd  Q_df  Li_fd  Co_fd
0    0.0   0.0    0.0    0.0
1    1.0   0.0    0.0    0.0
2    0.0   1.0    0.0    0.0
3    1.0   1.0    0.0    0.0
4    0.0   0.0    1.0    0.0
5    1.0   0.0    1.0    0.0
6    0.0   1.0    1.0    0.0
7    1.0   1.0    1.0    0.0
8    0.0   0.0    0.0    1.0
9    1.0   0.0    0.0    1.0
10   0.0   1.0    0.0    1.0
11   1.0   1.0    0.0    1.0
12   0.0   0.0    1.0    1.0
13   1.0   0.0    1.0    1.0
14   0.0   1.0    1.0    1.0
15   1.0   1.0    1.0    1.0
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,1].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,2].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,3].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08,

In [28]:
# Adding noise
Co_prdt = outputs_df["Q_Co_prdt"]
print(Co_prdt)

Co_prdt_avg = np.average(Co_prdt)
percent_response = 0.05
std = Co_prdt_avg*percent_response

noise = stats.norm.rvs(loc=0, scale=std, size=len(Co_prdt))
print(noise)

Q_noise = Co_prdt + noise
print(Q_noise)

0      3.54
1     23.54
2      9.54
3     29.54
4      3.54
5     23.54
6      9.54
7     29.54
8      3.54
9     23.54
10     9.54
11    29.54
12     3.54
13    23.54
14     9.54
15    29.54
Name: Q_Co_prdt, dtype: float64
[ 0.35433005 -0.68758062  1.11458496  0.69868602  0.76393734  0.11442898
 -0.00422064  0.02512274  0.64962612 -0.93346536  1.5345227   0.29246784
  0.42346635 -0.47107734 -0.37885796 -2.10229634]
0      3.894330
1     22.852419
2     10.654585
3     30.238686
4      4.303937
5     23.654429
6      9.535779
7     29.565123
8      4.189626
9     22.606535
10    11.074523
11    29.832468
12     3.963466
13    23.068923
14     9.161142
15    27.437704
Name: Q_Co_prdt, dtype: float64
